In [1]:
!pip install pandas
!pip install numpy
!pip install sklearn
!pip install transformers
!pip install demoji
!pip install googletrans==3.1.0a0


Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/share/pkg.7/python3/3.8.10/install/bin/python3.8 -m pip in

In [2]:
import re
from sklearn import metrics
from sklearn.metrics import classification_report

# BERT imports
import torch
# from urllib3.util.ssl_ import ssl
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler, TensorDataset
import pandas as pd
print(pd.__version__)
import io
import numpy as np
import unicodedata
from googletrans import Translator
import demoji
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from collections import defaultdict

1.2.4


## Define Preprocess Operations

In [3]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join( c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def trans_text(df, col):
    translator = Translator()
    translations = {}
    # translations[t] = translator.translate(t,dest='en').text
    cnt = 0
    df_en = df.copy()
    translations = {}
    # unique elements of the column
    unique_elements = df_en[col].unique()
    for element in unique_elements:
        # add translation to the dictionary
        detected_eq = translator.detect(element)
        if detected_eq.lang!='en' and detected_eq.confidence> 0.5:
            print(element)
            translations[element] = translator.translate(element).text
            cnt+=1
    return translations, int(cnt)

print(demoji.replace_with_desc('▪', " "))
translator = Translator()
try:
    trans = translator.translate('안녕하세요.▪',dest='en')
    translation = translator.translate("▪Der Himmel ist blau und ich mag Bananen", dest='en')
except BaseException as err:
    print('Error returned')
    print(f"Unexpected {err} =, {type(err)}")
print(translation.text)
print(trans.text)

 black small square 
▪ The sky is blue and I like bananas
Hello.▪


In [4]:

regexMap={r"<[\w'/'\s]*>": "",r"[\'\"\-]+": "",r"@[\w]+":"",r"#[\w]+":""}

def preprocess(datainput):
    t=datainput
    t = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', t, flags=re.MULTILINE)
    for regx in regexMap.keys():
        t = re.sub(regx, regexMap[regx], t)
    return t


def remove_emoji(datainput):
    t=datainput
    t = demoji.replace_with_desc(t, " ")
    return t


def lower_strip(datainput):
    t=datainput
    t = unicodeToAscii(t.strip())
    # Lowercase and trim
    return t

## Load Twitter Dataset Files:  Training & Test 

In [5]:
# Access the training & test data: 
twitter_train=pd.read_csv('./training.1600000.processed.noemoticon.csv',encoding='latin1',header=None)
twitter_test=pd.read_csv('./testdata.manual.2009.06.14.csv',encoding='latin1',header=None)

# Prep the training & test data: 
# Select the subset of data fields & name the columns
twitter_train=twitter_train[[5,0]]
twitter_train.columns=['tweet','sentiment', ]
twitter_test=twitter_test[[5,0]]
twitter_test.columns=['tweet','sentiment',]

## Downselect the Test data

In [6]:
# Down SELECT THE DATA - we have neutral sentiment data but not enough for a viable model

# #Switch out half of the no-emotion dataset in the test file for other data in the train file
# a = twitter_train[(twitter_train['sentiment']==0)][0:34]
# b = twitter_train[(twitter_train['sentiment']==4)][60:95]
# c = twitter_test[(twitter_test['sentiment']==2)][0:34]
# d = twitter_test[(twitter_test['sentiment']==2)][60:95]
# ai = twitter_train.index[(twitter_train['sentiment']==0)][0:34]
# bi = twitter_train.index[(twitter_train['sentiment']==4)][60:95]
# ci = twitter_test.index[(twitter_test['sentiment']==2)][0:34]
# di = twitter_test.index[(twitter_test['sentiment']==2)][60:95]
# twitter_train = twitter_train.drop(ai)
# twitter_train = twitter_train.drop(bi)
# twitter_train = twitter_train.append(c)
# twitter_train = twitter_train.append(d)
# twitter_test = twitter_test.drop(ci)
# twitter_test = twitter_test.drop(di)
# twitter_test = twitter_test.append(a)
# twitter_test = twitter_test.append(b)

# Try dropping the neutral sentiments from test per instructor's notes
ai = twitter_test.index[(twitter_test['sentiment']==2)]
twitter_test = twitter_test.drop(ai)

datasets=[twitter_train,twitter_test]
for i in datasets:
  print(len(i))

twitter_train['tweet'][6800]
# print(twitter_test['tweet'][200])

1600000
359


"The Red Shore and ent will make this Tuesday a little more enjoyable...but that's after class  Hope I make it..."

In [7]:
# Prep the training & test data: 
# remove usernames and URL links
datasets=[twitter_train,twitter_test]
for i in datasets:
    i['tweet']=i['tweet'].apply(preprocess)
    print(i.head())

                                               tweet  sentiment
0     Awww, thats a bummer.  You shoulda got Davi...          0
1  is upset that he cant update his Facebook by t...          0
2   I dived many times for the ball. Managed to s...          0
3    my whole body feels itchy and like its on fire           0
4   no, its not behaving at all. im mad. why am i...          0
                                               tweet  sentiment
0   I loooooooovvvvvveee my Kindle2. Not that the...          4
1  Reading my kindle2...  Love it... Lee childs i...          4
2  Ok, first assesment of the  ...it fucking rock...          4
3   Youll love your Kindle2. Ive had mine for a f...          4
4    Fair enough. But i have the Kindle2 and I th...          4


In [8]:
datasets=[twitter_train,twitter_test]
for i in datasets:
    i['tweet']=i['tweet'].apply(remove_emoji)
    i['tweet']=i['tweet'].apply(lower_strip)
#     i.to_csv('./lower_'+i+'.csv', header=None, index=False,encoding='utf-8')


In [9]:
twitter_train['tweet'][6800]

'The Red Shore and ent will make this Tuesday a little more enjoyable...but thats after class  Hope I make it...'

In [10]:
twitter_train.to_csv('./cased_train.csv', header=None, index=False,encoding="utf-8")
twitter_test.to_csv('./cased_test.csv', header=None, index=False,encoding="utf-8")

In [11]:
twitter_train=pd.read_csv('./cased_train.csv',encoding="utf-8",header=None)
twitter_test=pd.read_csv('./cased_test.csv',encoding="utf-8",header=None)
twitter_train=twitter_train[[0,1]]
twitter_train.columns=['tweet','sentiment', ]
twitter_test=twitter_test[[0,1]]
twitter_test.columns=['tweet','sentiment',]

# Google Translate takes too LONG

In [12]:
# Google translate the data

# final_non_english_total=0
# processed_rows=0
# batch_size = 80
# for i in range(0,len(twitter_train),batch_size):
# # for i in range(0,batch_size,batch_size):
#   translations, cnt = trans_text(twitter_train.iloc[i:i+batch_size,[0]],'tweet')
#   if cnt >0:
#       # modify all the terms of the data frame by using the translations dictionary
#       twitter_train.replace(translations, inplace = True)

#   # note the outcome
#   final_non_english_total+=cnt
#   processed_rows+=batch_size
#   if final_non_english_total % 10000 == 0:
#     print(f' processed so far {processed_rows}')

# # twitter_train['tweet'] = final_dataframe['tweet']
# print(f' total changes: {final_non_english_total}')
# print(f' processed : {processed_rows}')
# twitter_train.to_csv('/content/drive/My Drive/Colab Notebooks/trainingandtestdata/translated_train.csv', header=None, index=False,encoding='latin1')


# old buggy version
# xx=twitter_test[0:len(twitter_test)].copy()
# xx['tweet']=twitter_test['tweet'].apply(translate_text)
# twitter_train.replace(xx, inplace = True)
# print(twitter_test.head())

In [13]:
twitter_train['tweet'][400]

'woke up and was having an accident  &quot;Its pushing, its pushing!&quot; he was crying because he couldnt stop from wetting his pants.'

In [14]:
print(twitter_test['tweet'][350])
len(twitter_test[(twitter_test['sentiment']==2)])

getting ready to test out some burger receipes this weekend. Bobby Flay has some great receipes to try. Thanks Bobby.


0